In [1]:
import numpy as np
import cv2

In [53]:
def erod(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross' or type_of_mask == 'vertline' or type_of_mask == 'horizline'
    result = image.copy()
    if type_of_mask == 'square':
        result = erod_square(result, size)
    elif type_of_mask == 'cross':
        result = erod_cross(result, size)
    elif type_of_mask == 'vertline':
        result = erod_vert_line(result, size)
    elif type_of_mask == 'horizline':
        result = erod_horiz_line(result, size)
    return result

In [3]:
def erod_square(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i + k, j] == 0 and image[i - k, j] == 0 and image[i, j + k] == 0 
                and image[i + k, j + k] == 0 and image[i - k, j + k] == 0 and image[i, j - k] == 0 and image[i + k, j - k] == 0
                and image[i - k, j - k] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result

In [4]:
def erod_cross(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i + k, j] == 0 and image[i - k, j] == 0
                         and image[i, j + k] == 0 and image[i, j - k] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result

In [5]:
def dilat(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    if type_of_mask == 'square':
        result = dilat_square(result, size)
    elif type_of_mask == 'cross':
        result = dilat_cross(result, size)
    return result

In [6]:
def dilat_square(image, size):
    result = image.copy()
    status = False
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status or image[i + k, j] == 0 or image[i - k, j] == 0 or image[i, j + k] == 0
                         or image[i + k, j + k] == 0 or image[i - k, j + k] == 0 or image[i,j - k] == 0
                         or image[i + k, j - k] == 0 or image[i - k, j - k] == 0)
                if status:
                    break
            if status:
                result[i, j] = 0
            status = False
    return result

In [7]:
def dilat_cross(image, size):
    result = image.copy()
    status = False
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status or image[i + 1, j] == 0 or image[i - 1, j] == 0 or image[i, j + k] == 0
                         or image[i, j - k] == 0)
                if status:
                    break
            if status:
                result[i, j] = 0
            status = False
    return result

In [8]:
def opening(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = erod(result, size, type_of_mask)
    result = dilat(result, size, type_of_mask)
    return result

In [9]:
def closing(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = dilat(result, size, type_of_mask)
    result = erod(result, size, type_of_mask)
    return result

In [10]:
def filter(image):
    result = image.copy()
    for i in range(3, image.shape[0] - 3):
        for j in range(3, image.shape[1] - 3):
            if image[i, j] == 0:
                if image[i, j + 1] == 255 and image[i, j - 1] == 255 and image[i + 1, j] == 255 and image[i - 1, j] == 255:
                    image[i, j] = 255
            elif image[i, j] == 255:
                if image[i, j + 1] == 0 and image[i, j - 1] == 0 and image[i + 1, j] == 0 and image[i - 1, j] == 0:
                    image[i, j] = 0
    return result

In [58]:
def make_noise(image, prob):
    result = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            p = np.random.randint(0, 101)
            if p < prob :
                if image[i, j] == 0:
                    result[i, j] = 255
                elif image[i, j] == 255:
                    result[i, j] = 0
    return result

In [12]:
def circle(image, centre, radius):
    circle = [centre, radius]
    for i in range(circle[0][0] - circle[1], circle[0][0] + circle[1]):
        for j in range(circle[0][1] - circle[1], circle[0][1] + circle[1]):
            if (i - circle[0][0]) ** 2 + (j - circle[0][1]) ** 2 < circle[1] ** 2:
                image[i, j] = 0

In [13]:
def triangle(image, A, B, C):
    triangle = [A, B, C]
    x_a = triangle[0][1]
    y_a = triangle[0][0]
    x_b = triangle[1][1]
    y_b = triangle[1][0]
    x_c = triangle[2][1]
    y_c = triangle[2][0]

    left = triangle[0][0]
    right = triangle[0][0]
    above = triangle[0][1]
    below = triangle[0][1]
    for i in triangle:
        if left > i[0]:
            left = i[0]
        if right < i[0]:
            right = i[0]
        if above > i[1]:
            above = i[1]
        if below < i[1]:
            below = i[1]
    for i in range(above, below):
        for j in range(left, right):
            coord_1 = (x_a - i) * (y_b - y_a) - (x_b - x_a) * (y_a - j)
            coord_2 = (x_b - i) * (y_c - y_b) - (x_c - x_b) * (y_b - j)
            coord_3 = (x_c - i) * (y_a - y_c) - (x_a - x_c) * (y_c - j)
            if coord_1 >= 0 and coord_2 >= 0 and coord_3 >= 0:
                image[i, j] = 0

In [14]:
def square(image, A, B):
    square = [A, B]
    for i in range(square[0][0], square[1][0]):
        for j in range(square[0][1], square[1][1]):
            image[i, j] = 0

In [15]:
def noise_reduce(image, filter_image, noised_image):
    assert image.shape == filter_image.shape and image.shape == noised_image.shape
    count_noised = 0
    count_filtered = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i, j] - noised_image[i, j] != 0:
                count_noised += 1
            if image[i, j] - filter_image[i, j] != 0:
                count_filtered += 1
    return count_noised / count_filtered

# 1. Фильтрация бинарных изображений

## 1.1. Создать или выбрать бинарное изображение с любым простым рисунком, состоящим из примитивных фигур (круги, квадраты, полигоны).

In [85]:
image = np.zeros((500, 500))
image.fill(255)

In [86]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [87]:
cv2.imwrite('/home/qwerty/image.png', image)

True

## 1.2. На полученное изображение наложить шум (1%, 2%, 5%, 10%, 20%, 50%).

In [88]:
noised = []
for i in [1, 2, 5, 10, 20, 50]:
    noise = make_noise(image, i)
    noised.append(noise)
    cv2.imwrite('/home/qwerty/image_noise_' + str(i) + '.png', noise)

## 1.3. Выполнить операции эрозии и дилатации для различных структурных элементов (крест, квадрат). Визуально оценить результаты эрозии и дилатации.

In [89]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'square')
    cv2.imwrite('/home/qwerty/erod_image_square_' + str(i) + '.png', erod_image)

In [90]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'cross')
    cv2.imwrite('/home/qwerty/erod_image_cross_'+ str(i) + '.png', erod_image)

In [91]:
for i in range(3, 6):
    dilat_image = dilat(image, i, type_of_mask = 'square')
    cv2.imwrite('/home/qwerty/dilat_image_square_' + str(i) + '.png', dilat_image)

In [92]:
for i in range(3, 6):
    dilat_image= dilat(image, i, type_of_mask = 'cross')
    cv2.imwrite('/home/qwerty/dilat_image_cross_' + str(i) + '.png', dilat_image)

## 1.4. Провести фильтрацию шумов на изображении при помощи морфологических операций вскрытия и закрытия с различными размерами (от 3 до 5) и формами структурного элемента (крест, квадрат), а также алгоритма фильтрации шумов, которому соответствует логическая таблица истинности, представленная в табл. 1 со крестообразным структурным элементом размером 3.

In [93]:
opened_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'square')
        opened_image_square[i - 3, j] = open_image
        cv2.imwrite('/home/qwerty/open_image_square' + str(j) + '_' + str(i) + '.png', open_image)

In [94]:
closed_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'square')
        closed_image_square[i - 3, j] = close_image
        cv2.imwrite('/home/qwerty/close_image_square' + str(j) + '_' + str(i) + '.png', close_image)

In [95]:
opened_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'cross')
        opened_image_cross[i - 3, j] = open_image
        cv2.imwrite('/home/qwerty/open_image_cross' + str(j) + '_' + str(i) + '.png', open_image)

In [96]:
closed_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'cross')
        closed_image_cross[i - 3, j] = close_image
        cv2.imwrite('/home/qwerty/close_image_cross' + str(j) + '_' + str(i) + '.png', close_image)

In [97]:
filtered_image = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(len(noised)):
    filter_image = filter(noised[i])
    filtered_image[i - 3, j] = filter_image
    cv2.imwrite('/home/qwerty/filter_image_' + str(i) + '.png', filter_image)

## 1.5. Рассчитать коэффициент снижения шума для различных методов фильтрации, для различной интенсивности шума и различных размеров и форм структурного элемента (окна).

In [98]:
for i in range(opened_image_square.shape[0]):
    for j in range(opened_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 0.7065637065637066
Size 3 Noised 1 reduce 1.0074349442379182
Size 3 Noised 2 reduce 1.7418060200668897
Size 3 Noised 3 reduce 2.6598480420806547
Size 3 Noised 4 reduce 3.5762424242424244
Size 3 Noised 5 reduce 6.8838967971530245
Size 4 Noised 0 reduce 0.6654545454545454
Size 4 Noised 1 reduce 0.9344827586206896
Size 4 Noised 2 reduce 1.7199471598414795
Size 4 Noised 3 reduce 2.6087704213241616
Size 4 Noised 4 reduce 3.5148915892304027
Size 4 Noised 5 reduce 6.712937859234357
Size 5 Noised 0 reduce 0.3560311284046693
Size 5 Noised 1 reduce 0.5784418356456777
Size 5 Noised 2 reduce 1.0280300039478878
Size 5 Noised 3 reduce 1.575831024930748
Size 5 Noised 4 reduce 2.2610161698214424
Size 5 Noised 5 reduce 6.927811975377728


In [99]:
for i in range(closed_image_square.shape[0]):
    for j in range(closed_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 0.07093023255813953
Size 3 Noised 1 reduce 0.09742944454431063
Size 3 Noised 2 reduce 0.15329369517866603
Size 3 Noised 3 reduce 0.19494120922661756
Size 3 Noised 4 reduce 0.24071928593574074
Size 3 Noised 5 reduce 0.5346739050629905
Size 4 Noised 0 reduce 0.07104037267080746
Size 4 Noised 1 reduce 0.09749955027882713
Size 4 Noised 2 reduce 0.1535377358490566
Size 4 Noised 3 reduce 0.19531769704512777
Size 4 Noised 4 reduce 0.24137902823342688
Size 4 Noised 5 reduce 0.5358402693917477
Size 5 Noised 0 reduce 0.07290836653386454
Size 5 Noised 1 reduce 0.10296352583586627
Size 5 Noised 2 reduce 0.1419924750531654
Size 5 Noised 3 reduce 0.1401364105248572
Size 5 Noised 4 reduce 0.1661074203355478
Size 5 Noised 5 reduce 0.5336989041497461


In [100]:
for i in range(opened_image_cross.shape[0]):
    for j in range(opened_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 0.8970588235294118
Size 3 Noised 1 reduce 1.3618090452261307
Size 3 Noised 2 reduce 2.570582428430405
Size 3 Noised 3 reduce 4.414161008729389
Size 3 Noised 4 reduce 5.529235382308846
Size 3 Noised 5 reduce 2.9741741741741743
Size 4 Noised 0 reduce 0.8472222222222222
Size 4 Noised 1 reduce 1.266355140186916
Size 4 Noised 2 reduce 2.48
Size 4 Noised 3 reduce 4.251284446520318
Size 4 Noised 4 reduce 5.374134790528233
Size 4 Noised 5 reduce 2.9487423780487805
Size 5 Noised 0 reduce 0.3160621761658031
Size 5 Noised 1 reduce 0.6537997587454765
Size 5 Noised 2 reduce 1.552772808586762
Size 5 Noised 3 reduce 2.718637992831541
Size 5 Noised 4 reduce 3.607728050868183
Size 5 Noised 5 reduce 6.755798090040928


In [101]:
for i in range(closed_image_cross.shape[0]):
    for j in range(closed_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 0.07555739058629232
Size 3 Noised 1 reduce 0.11172954030096888
Size 3 Noised 2 reduce 0.2081035722848238
Size 3 Noised 3 reduce 0.32458455174381284
Size 3 Noised 4 reduce 0.4200335981321716
Size 3 Noised 5 reduce 0.5985186904140318
Size 4 Noised 0 reduce 0.07568238213399504
Size 4 Noised 1 reduce 0.11172954030096888
Size 4 Noised 2 reduce 0.20830333573314136
Size 4 Noised 3 reduce 0.3249321719263173
Size 4 Noised 4 reduce 0.4204705781755476
Size 4 Noised 5 reduce 0.5997248435289786
Size 5 Noised 0 reduce 0.4906166219839142
Size 5 Noised 1 reduce 1.0524271844660193
Size 5 Noised 2 reduce 1.4022617124394183
Size 5 Noised 3 reduce 0.910382076415283
Size 5 Noised 4 reduce 0.5204904295669048
Size 5 Noised 5 reduce 0.5654388088332686


In [102]:
for i in range(filtered_image.shape[0]):
    for j in range(filtered_image.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, filtered_image[i, j], noised[j])))

Size 3 Noised 0 reduce 0.0007681426478674597
Size 3 Noised 1 reduce 0.002275045437946247
Size 3 Noised 2 reduce 0.01093029210408123
Size 3 Noised 3 reduce 0.03820565235458808
Size 3 Noised 4 reduce 0.12384306383978978
Size 3 Noised 5 reduce 4.961724980962686
Size 4 Noised 0 reduce 0.0007681426478674597
Size 4 Noised 1 reduce 0.002275045437946247
Size 4 Noised 2 reduce 0.01093029210408123
Size 4 Noised 3 reduce 0.03820565235458808
Size 4 Noised 4 reduce 0.12384306383978978
Size 4 Noised 5 reduce 2.507900494287335
Size 5 Noised 0 reduce 0.0007681426478674597
Size 5 Noised 1 reduce 0.002275045437946247
Size 5 Noised 2 reduce 0.01093029210408123
Size 5 Noised 3 reduce 0.03820565235458808
Size 5 Noised 4 reduce 0.12384306383978978
Size 5 Noised 5 reduce 0.9983709940162255


# 2. Выделение контуров на бинарных изображениях

## 2.1. Создать или выбрать бинарное изображение, содержащее простой рисунок, состоящий из примитивных фигур (полосы, круги, квадраты).

In [103]:
image = np.zeros((500, 500))
image.fill(255)

In [104]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [105]:
cv2.imwrite('/home/qwerty/image.png', image)

True

## 2.2. С помощью морфологических операций выделить контур объекта. Выяснить, когда контур получается внешним, внутренним, четырёхсвязным, восьмисвязным.

In [106]:
def border(image, mask_type):
    assert mask_type == 'square' or mask_type == 'cross' or mask_type == 'vertline' or mask_type == 'horizline'
    
    result = image.copy()
    erod_image = erod(image, 3, mask_type)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if image[i, j] - erod_image[i, j] != 0:
                result[i, j] = 0
            else:
                result[i, j] = 255
    return result

In [107]:
b = border(image, 'square')
cv2.imwrite('/home/qwerty/b.png', b)

True

In [108]:
def erod_vert_line(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i, j - k] == 0 and image[i, j + k] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result

In [109]:
b = border(image, 'vertline')
cv2.imwrite('/home/qwerty/b.png', b)

True

In [110]:
def erod_horiz_line(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i - 1, j] == 0 and image[i + 1, j] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result